In [ ]:
# Importing all necessary libraries

In [1]:
import requests as re
from bs4 import BeautifulSoup as BS
import re as r
import os
import math


In [2]:
# Function for extracting the URL for articles in Egemen.kz

In [12]:
def extract_urls(base_url,search_url):
    response = re.get(search_url)
    html_content = response.text
    
    # Парсинг HTML
    soup = BS(html_content, "html.parser")
    
    divs = soup.find_all('div', class_='clearfix news-t flexBlock')

    # Extract URLs from <a> tags inside those divs
    urls = [div.a['href'] for div in divs if div.a]
    full_urls = [base_url.rstrip('/') + div.a['href'] for div in divs if div.a]

    return full_urls

In [13]:
# Created the function specifically for the parsing Egemen.kz web-site content and saved as a text file
# Only works for one URL 

In [14]:
def article_contents(article_url):
    
    # Загрузка HTML страницы
    response = re.get(article_url)
    html_content = response.text
    
    # Парсинг HTML
    soup = BS(html_content, "html.parser")
    
    # Title extract
    title = soup.find('h1')
    title = title.get_text(strip=True)
    
    # Content extract
    article_body = soup.find("div", itemprop="articleBody")

    # Extract and clean all text from the div
    if article_body:
        article_text = article_body.get_text(separator="\n", strip=True)
    else:
        print("Article body not found!")
    
    # Сохранение текста в файл

    valid_title = r.sub(r'[\\/:"*?<>|]+', '', title)  # Remove invalid characters
    filename = f"{valid_title}.txt"


    with open(filename, "w", encoding="utf-8") as file:
        file.write(f"{title}\n\n")
        file.write(article_text)

    print("Статья успешно сохранена!")
    

In [20]:
def main():
    
    # Key word input
    key_word_input = input("Введите ключевое слово: \n")
    
    # Creating url with key-word
    base_url_search = "https://egemen.kz"                         #base url canbe changed by other URL
    key_word_url = base_url_search + "/search?q=" + key_word_input 

    print(f"Ссылка по вашему запросу: \n{key_word_url}")

    response = re.get(key_word_url) 
    html_content = response.text
    
    # Parsing HTML
    soup = BS(html_content, "html.parser")
    
    # Finding articles number, included in web-site
    article_founded = soup.find('small').text

    print(article_founded)
    
    # Conversion to int from list
    num_article = r.findall(r'\d+', article_founded)
    num = int(num_article[0]) 
    
    # Each web-page in site only contains 5 articels, it can be also changed 
    articles_per_page = 5
    pages = math.ceil(num / articles_per_page) 

    print(f"Чило страниц:{pages}")
    
    urls_list = []  # Initialize the list

    for page in range(1, pages + 1):
        full_url = key_word_url + "&page=" + str(page)
        print(f"Processing page: {page} {full_url}\n")
    
        # Call the function and get the extracted URLs
        article_urls = extract_urls(base_url_search, full_url)
    
        for article_url in article_urls: 
            print(f"{article_url}\n")    
    
        # Append URLs to the list
        urls_list.extend(article_urls) 
    
        # Skip if no URLs are found
        if not article_urls:  
            print(f"Skipping page {page} due to no articles found.\n")
        continue

    # The final list of URLs
    print("\nAll Extracted URLs:\n")

    for url in urls_list:
        print(url)
    
    print(f"\nCount:{len(urls_list)}\n")
    print("Starting saving all the extracted articles to the text")
    for url in urls_list:
        print(url)
        article_contents(url)

In [21]:
if __name__ == "__main__":
    main()

Введите ключевое слово: 
Донор
Ссылка по вашему запросу: 
https://egemen.kz/search?q=Донор
73 материал табылды
Чило страниц:15
Processing page: 1 https://egemen.kz/search?q=Донор&page=1

https://egemen.kz/article/377917-mayittik-donorlyqty-damytugha-ne-kedergi

https://egemen.kz/article/375100-qaytys-bolghan-donordynh-aghza-musheleri-4-adamnynh-omirin-saqtap-qaldy

https://egemen.kz/article/373373-suyek-kemigi-donory

https://egemen.kz/article/370779-bes-adamgha-omir-syylaghan-donor

https://egemen.kz/article/367937-donor-buyrekpen-dgurip-dombyra-unin-kokke-samghatty

Processing page: 2 https://egemen.kz/search?q=Донор&page=2

https://egemen.kz/article/367714-donor-oblys-qatary-qaytse-artady

https://egemen.kz/article/357191-mayit-donorlyghyn-qalpyna-keltiru-qadget-eks-ministr-transplantatsiyadaghy-tuytk

https://egemen.kz/article/356281-eki-donor-toghyz-adamnynh-omirin-saqtady-ministrlik-donorlyqqa-kelisken-azamatta

https://egemen.kz/article/356279-aghza-mushelerimdi-donor-retinde-be

Статья успешно сохранена!
https://egemen.kz/article/375100-qaytys-bolghan-donordynh-aghza-musheleri-4-adamnynh-omirin-saqtap-qaldy
Статья успешно сохранена!
https://egemen.kz/article/373373-suyek-kemigi-donory
Статья успешно сохранена!
https://egemen.kz/article/370779-bes-adamgha-omir-syylaghan-donor
Статья успешно сохранена!
https://egemen.kz/article/367937-donor-buyrekpen-dgurip-dombyra-unin-kokke-samghatty
Статья успешно сохранена!
https://egemen.kz/article/367714-donor-oblys-qatary-qaytse-artady
Статья успешно сохранена!
https://egemen.kz/article/357191-mayit-donorlyghyn-qalpyna-keltiru-qadget-eks-ministr-transplantatsiyadaghy-tuytk
Статья успешно сохранена!
https://egemen.kz/article/356281-eki-donor-toghyz-adamnynh-omirin-saqtady-ministrlik-donorlyqqa-kelisken-azamatta
Статья успешно сохранена!
https://egemen.kz/article/356279-aghza-mushelerimdi-donor-retinde-beruge-kelisemin-densaulyq-saqtau-ministri
Статья успешно сохранена!
https://egemen.kz/article/351147-shymkent-donor-shaqar